In [1]:
import duckdb

# Caminhos para os bancos de dados
db1_path = 'fineweb.duckdb'  # Substitua pelo caminho do seu primeiro banco de dados
db2_path = 'books.duckdb'  # Substitua pelo caminho do seu segundo banco de dados

# Alias para o segundo banco de dados
alias_db2 = 'db2_alias'

# Conectar ao banco de dados principal
conn = duckdb.connect(database=db1_path, read_only=False)

# Anexar o segundo banco de dados
conn.execute(f"ATTACH DATABASE '{db2_path}' AS {alias_db2}")

# Verificar esquemas (opcional, mas recomendado)
schema_main = conn.execute("DESCRIBE dataset").fetchdf()
schema_db2 = conn.execute(f"DESCRIBE {alias_db2}.dataset").fetchdf()

if not schema_main.equals(schema_db2):
	raise Exception("Os esquemas das tabelas 'dataset' nos dois bancos de dados não são compatíveis.")

# Executar a consulta modificada
df_training = conn.execute(f"""
WITH combined_dataset AS (
    SELECT * FROM dataset
    UNION ALL
    SELECT * FROM {alias_db2}.dataset
),
sampled_a AS (
    SELECT id, indice, content, name
    FROM (
        SELECT
            id,
            indice,
            content,
            name,
            ROW_NUMBER() OVER (PARTITION BY name ORDER BY RANDOM()) AS rn
        FROM combined_dataset
    ) sub
    WHERE rn <= 1
),
sampled_b AS (
    SELECT id, indice, content, name
    FROM (
        SELECT
            id,
            indice,
            content,
            name,
            ROW_NUMBER() OVER (PARTITION BY name ORDER BY RANDOM()) AS rn
        FROM combined_dataset
    ) sub
    WHERE rn <= 1
)
SELECT
    a.name AS name,
    a.content AS content1,
    b.content AS content2,
    -- Cálculo original do target_transformado
    SIGN(a.indice - b.indice) * LN(1 + ABS(a.indice - b.indice)) AS target_transformed,
    -- Aplicação da função sigmoide no target_transformado
    1 / (1 + EXP(- (SIGN(a.indice - b.indice) * LN(1 + ABS(a.indice - b.indice))))) AS target_transformed_sigmoid
FROM sampled_a a
JOIN sampled_b b
    ON a.name = b.name
ORDER BY RANDOM()
LIMIT 50000;
""").df()

conn.close()

df_training

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,name,content1,content2,target_transformed,target_transformed_sigmoid
0,https://www.uihealthcare.org/2column.aspx?id=2...,Samuel. They were with me 100% of the way. The...,"a curable disease of my right hip. Of course, I",7.014814,0.999102
1,http://secularright.org/,its videos. How selfish! Clearly the fledgelin...,"though with the expected qualifier,",8.074960,0.999689
2,https://www.poemhunter.com/poem/blood-is-red/,Blood Is Red - Poem by nish dawn Blood is,"You pushed me, I am",-5.313206,0.004902
3,http://www.jinx.com/p/portal_2_hi_five_premium...,of insane tests at the behest of an unseen tor...,while the test robots would,-5.924256,0.002667
4,http://www.dhammawheel.com/viewtopic.php?p=225103,"house was, what the danger","each felt emboldened in heart and, pushing and...",-6.974479,0.000935
...,...,...,...,...,...
8698,http://www.postandcourier.com/article/20130909...,"to article © 2016,",ERROR: Macro postandcourier/header is missing!...,4.624973,0.990291
8699,https://support.yet-another-mail-merge.com/hc/...,"can help you improve your business, and become...","can help you improve your business, and become...",0.000000,0.500000
8700,http://www.marinadiportorotondo.it/en/berths,"and of berths, the respect","the ""Spiaggia Rosa"" (pink",5.288267,0.994975
8701,https://graceoc.com/announcements/pj-announcem...,"a day to connect, pray, worship, and be encour...",beginning April 20th. Click,6.546785,0.998567


In [5]:
import torch

for _, row in df_training.iterrows():
	print([row['content1'],row['content2']])

	encode = embedding_generator.tokenizer(
			[row['content1'],row['content2']],
			return_tensors="pt",
			padding=True,  # Padding dinâmico
			truncation=True)
  
	embeddings1 = embedding_generator.weights[encode['input_ids'][0]]
	embeddings2 = embedding_generator.weights[encode['input_ids'][1]]

	embeddings = torch.stack([embeddings1, embeddings2], dim=0)  # (batch_size, 2, seq_length, vector_size)
	
	lengths1 = encode['attention_mask'][0].sum()
	lengths2 = encode['attention_mask'][1].sum()
	lengths = torch.stack([lengths1, lengths2])  # (batch_size, 2)

	targets = torch.tensor(row['target_transformed'], dtype=torch.float16)  # (batch_size,)

	print(embeddings.shape)
	break

["facilitate the acquisition of perspective in the student's chosen area", 'selective in order for the']
torch.Size([2, 14, 2048])


In [ ]:
import matplotlib.pyplot as plt

plt.hist(df_training['target_transformed'], bins=50, alpha=0.5, label='target_transformed')
plt.legend()
plt.show()

In [ ]:
# Cria um iterador a partir do dataset
train_iter = iter(validation_dataloader)

# Obtém a próxima amostra
embeddings, lengths, target = next(train_iter)

# Exibe a amostra
print("Embeddings:", embeddings.shape)
print("Lengths:", lengths)
print("Target:", target)

In [8]:
#  Converter DataLoader para iterador
data_iter = iter(validation_dataloader)
# Pegar o primeiro lote
embeddings, lengths, targets = next(data_iter)
embeddings.shape

collate_fn 150


torch.Size([150, 2, 61, 2048])

In [9]:
# Pegar o primeiro lote
embeddings, lengths, targets = next(data_iter)
embeddings.shape

_fetch_data
_fetch_data iterrows
_fetch_data Tokenize
Converter targets
collate_fn 1


torch.Size([1, 2, 24, 2048])

In [1]:
import torch
import gc

if not torch.cuda.is_available():
    print("CUDA is not available. Please ensure you have a compatible GPU and drivers installed.")
else:
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")

Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [1]:
import torch
import numpy as np
from transformers import AutoTokenizer
import os

# Definir paralelismo corretamente
os.environ["TOKENIZERS_PARALLELISM"] = "true"

class TextToEmbedding:
    def __init__(self, weights_path, num_ids=128256, vector_size=2048, device='cpu'):
        """
        Inicializa a classe TextToEmbedding sem padding fixo.

        Args:
            weights_path (str): Caminho para o arquivo .npy que contém os pesos.
            num_ids (int, opcional): Número total de IDs. Padrão é 128256.
            vector_size (int, opcional): Tamanho de cada vetor de embedding. Padrão é 2048.
            device (str, opcional): Dispositivo para carregar os tensores ('cpu' ou 'cuda'). Padrão é 'cpu'.
        """
		
        self.device = device

        # Carrega o tokenizer sem padding fixo
        self.tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B", use_fast=True)
        
        # Carrega os pesos a partir do arquivo .npy
        try:
            weights_np = np.load(weights_path)
            self.weights = torch.from_numpy(weights_np).to(self.device)
        except FileNotFoundError:
            raise FileNotFoundError(f"O arquivo de pesos '{weights_path}' não foi encontrado.")
        except Exception as e:
            raise RuntimeError(f"Erro ao carregar os pesos: {e}")
        
        # Verifica a forma dos pesos
        if self.weights.shape != (num_ids, vector_size):
            raise ValueError(f"O formato do arquivo weights.npy é {self.weights.shape}, mas era esperado {(num_ids, vector_size)}.")

embedding_generator = TextToEmbedding("weights_half.npy", device='cuda')

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset

class SequenceDataset_val(Dataset):
	def __init__(self, df):
		self.df = df.reset_index(drop=True)

		# Tokenize os textos com padding dinâmico
		encoding1 = embedding_generator.tokenizer(
			list(df['content1']),
			return_tensors="pt",
			padding=True,  # Padding dinâmico
			truncation=True
		)
		
		encoding2 = embedding_generator.tokenizer(
			list(df['content2']),
			return_tensors="pt",
			padding=True,  # Padding dinâmico
			truncation=True
		)
		
		# Converter para o dispositivo
		input_ids1 = encoding1.input_ids.to(embedding_generator.device)
		attention_mask1 = encoding1.attention_mask.to(embedding_generator.device)
		input_ids2 = encoding2.input_ids.to(embedding_generator.device)
		attention_mask2 = encoding2.attention_mask.to(embedding_generator.device)
		
		# Obter embeddings
		embeddings1 = embedding_generator.weights[input_ids1]
		embeddings2 = embedding_generator.weights[input_ids2]
		
		# Concatenar as duas sequências no eixo 1
		self.embeddings = torch.stack([embeddings1, embeddings2], dim=1)  # (batch_size, 2, seq_length, vector_size)
		
		# Calcular comprimentos
		lengths1 = attention_mask1.sum(dim=1)
		lengths2 = attention_mask2.sum(dim=1)
		self.lengths = torch.stack([lengths1, lengths2], dim=1)  # (batch_size, 2)

		# Converter targets
		self.targets = torch.tensor(df['target_transformed'], dtype=torch.float16)  # (batch_size,)

	def __len__(self):
		return len(self.df)
	
	def __getitem__(self,idx):
		return self.embeddings[idx], self.lengths[idx], self.targets[idx]

In [3]:
import torch
from torch.utils.data import IterableDataset, DataLoader
import duckdb
from tqdm import tqdm
import pandas as pd

# Paths to the databases
db1_path = 'fineweb.duckdb'
db2_path = 'books.duckdb'

class IterableSequenceDataset(IterableDataset):
	def __init__(self, db1_path, db2_path, batch_size=100):
		super(IterableSequenceDataset, self).__init__()
		self.db1_path = db1_path
		self.db2_path = db2_path
		self.batch_size = batch_size

	def _fetch_data(self, conn):
		query = f"""
			WITH combined_dataset AS (
				SELECT * FROM main.dataset
				UNION ALL
				SELECT * FROM db2.dataset
			),
			sampled_a AS (
				SELECT id, indice, content, name
				FROM (
					SELECT
						id,
						indice,
						content,
						name,
						ROW_NUMBER() OVER (PARTITION BY name ORDER BY RANDOM()) AS rn
					FROM combined_dataset
				) sub
				WHERE rn <= 1
			),
			sampled_b AS (
				SELECT id, indice, content, name
				FROM (
					SELECT
						id,
						indice,
						content,
						name,
						ROW_NUMBER() OVER (PARTITION BY name ORDER BY RANDOM()) AS rn
					FROM combined_dataset
				) sub
				WHERE rn <= 1
			)
			SELECT
				a.name AS name,
				a.content AS content1,
				b.content AS content2,
				SIGN(a.indice - b.indice) * LN(1 + ABS(a.indice - b.indice)) AS target_transformed
			FROM sampled_a a
			JOIN sampled_b b
				ON a.name = b.name
			ORDER BY RANDOM()
			LIMIT {self.batch_size};
		"""
		print("_fetch_data")
		return conn.execute(query).df()

	def __iter__(self):
		"""
		Iterator que gera dados continuamente.

		Yields:
			tuple: (content1, content2, target) para cada amostra.
		"""
		# Estabelecer conexão com DuckDB
		conn = duckdb.connect(database=self.db1_path, read_only=True)
		conn.execute("SET enable_progress_bar=false")
		
		# Anexar o segundo banco de dados com um alias 'db2'
		conn.execute(f"ATTACH '{self.db2_path}' AS db2")

		try:
			while True:
				df = self._fetch_data(conn)
				for _, row in df.iterrows():
					
					print("_fetch_data iterrows")
					# Tokenize os textos com padding dinâmico
					encoding = embedding_generator.tokenizer(
						[row['content1'],row['content2']],
						return_tensors="pt",
						padding=True,  # Padding dinâmico
						truncation=True
					)
					
					# Converter para o dispositivo
					input_ids1 = encoding.input_ids[0].to(embedding_generator.device)
					attention_mask1 = encoding.attention_mask[0].to(embedding_generator.device)
					input_ids2 = encoding.input_ids[1].to(embedding_generator.device)
					attention_mask2 = encoding.attention_mask[1].to(embedding_generator.device)
					
					# Obter embeddings
					embeddings1 = embedding_generator.weights[input_ids1]
					embeddings2 = embedding_generator.weights[input_ids2]
					
					# Concatenar as duas sequências no eixo 1
					embeddings = torch.stack([embeddings1, embeddings2], dim=0)  # (batch_size, 2, seq_length, vector_size)
					
					# Calcular comprimentos
					lengths1 = attention_mask1.sum()
					lengths2 = attention_mask2.sum()
					lengths = torch.stack([lengths1, lengths2])  # (batch_size, 2)
					
					# Converter targets
					targets = torch.tensor(row['target_transformed'], dtype=torch.float16)  # (batch_size,)
				
					yield embeddings, lengths, targets

		finally:
			# Garantir que a conexão seja fechada quando o iterador for finalizado
			conn.close()

In [4]:
import numpy as np

def signed_log_transform(y):
    return np.sign(y) * np.log1p(np.abs(y))

def inverse_signed_log_transform(y_transformed):
    return np.sign(y_transformed) * (np.expm1(np.abs(y_transformed)))

def dissimilaridade(S):
	epsilon = 0
	if S == 0:
		max_x = np.log(np.finfo(np.float32).max)
		epsilon = (1/(max_x-1))
        
	return (1 - (S+epsilon)) / (S+epsilon)

In [5]:
import pandas as pd

data = {
    'target_transformed': [1.0]*50,
    "content1": ["A tempestade durou várias horas durante a noite.", "Ele estudou intensamente para o exame.", "A fábrica reduziu a emissão de poluentes.", "Maria começou a praticar exercícios regularmente.", "O motor do carro parou de funcionar.", "Houve uma forte seca na região.", "Ela começou a dormir melhor.", "A escola adotou uma alimentação saudável.", "O projeto teve apoio governamental.", "A internet caiu durante a reunião.", "Ele começou a economizar dinheiro mensalmente.", "A estrada estava em péssimo estado de conservação.", "O sistema de ar condicionado foi desligado no escritório.", "A empresa investiu em marketing digital.", "Ana não revisou o relatório antes de enviar.", "As crianças brincaram no parque até tarde.", "O curso de capacitação foi oferecido aos funcionários.", "Ele comprou um celular novo com câmera de alta qualidade.", "A reforma no prédio foi concluída.", "A região teve um crescimento populacional rápido.", "Carla começou a meditar diariamente.", "O atleta intensificou seu treino antes da competição.", "O sinal de celular na região foi ampliado.", "As temperaturas caíram drasticamente durante o inverno.", "Ele não configurou o alarme antes de dormir.", "A empresa ofereceu benefícios extras para seus funcionários.", "Houve um vazamento de gás na cozinha do restaurante.", "A biblioteca da escola foi modernizada com novos livros e tecnologia.", "Marcos passou a fazer pausas regulares durante o expediente.", "A empresa desenvolveu um aplicativo intuitivo para clientes.", "Devido ao aumento das chuvas nas últimas semanas, o nível dos rios subiu rapidamente e ultrapassou a capacidade das barragens.", "Ela decidiu começar uma rotina de alimentação balanceada, exercícios físicos regulares e meditação diária para melhorar sua saúde física e mental.", "A empresa, que sofria com baixos índices de produtividade, implementou uma nova estratégia de gestão focada no desenvolvimento dos funcionários e na cultura organizacional.", "O governo lançou um programa nacional de reciclagem e incentivou a participação ativa dos cidadãos por meio de campanhas de conscientização em escolas, empresas e residências.", "Após uma longa estiagem que afetou grande parte do território agrícola do país, o governo implementou um pacote emergencial de apoio aos agricultores, incluindo subsídios e incentivos para a recuperação das lavouras.", "Ele deixou o celular carregando a noite inteira sem usar carregadores de segurança e em um local sem ventilação.", "A comunidade local se uniu para limpar e revitalizar a praça abandonada, que estava sem manutenção há anos e havia se tornado um ponto de descarte irregular de lixo.", "Durante a reforma do prédio, descobriu-se que a estrutura tinha falhas graves, e o prazo para conclusão foi ampliado em seis meses para garantir a segurança.", "Um furacão atingiu a região costeira do país, com ventos de mais de 200 km/h, causando destruição em várias cidades e deixando milhares de pessoas desabrigadas.", "Ela esqueceu de regar a planta de sua varanda durante o mês todo, e o clima estava seco.", "João quebrou o braço jogando futebol.", "Devido a uma combinação de fatores climáticos extremos, incluindo ventos fortes e chuvas intensas, a região sofreu graves danos estruturais e interrupção no fornecimento de energia elétrica por vários dias.", "Ela esqueceu de levar o guarda-chuva.", "A empresa adotou práticas sustentáveis em sua produção, reduzindo o uso de plástico e implementando programas de reciclagem.", "A criança se recusou a comer legumes durante toda a semana.", "Após anos de pesquisa e desenvolvimento, a equipe científica finalmente descobriu um método eficiente para produzir energia limpa a partir de fontes renováveis.", "O café da manhã foi esquecido.", "A iniciativa comunitária organizou mutirões de limpeza nas praias locais regularmente durante o verão.", "Ele atualizou seu currículo e participou de várias entrevistas de emprego nos últimos meses.", "O artista decidiu experimentar novas técnicas em suas pinturas, incorporando elementos digitais e materiais reciclados em suas obras."],
    "content2": ["Pela manhã, muitas ruas estavam alagadas.", "Conseguiu uma nota alta na prova.", "A qualidade do ar na cidade melhorou.", "Ela perdeu peso e aumentou sua energia.", "Ele teve que chamar o guincho para levar o carro à oficina.", "As plantações foram prejudicadas, e a colheita foi menor.", "Sua disposição durante o dia melhorou significativamente.", "Os alunos passaram a ter mais energia e melhor concentração.", "Conseguiu concluir as fases iniciais rapidamente.", "A comunicação com a equipe foi interrompida.", "Conseguiu juntar uma quantia para uma viagem.", "O trânsito ficou mais lento e perigoso para os motoristas.", "O ambiente ficou quente e desconfortável para os funcionários.", "As vendas aumentaram significativamente.", "O documento continha erros e precisou ser corrigido.", "Elas ficaram cansadas e dormiram rapidamente ao chegar em casa.", "Eles melhoraram suas habilidades e eficiência no trabalho.", "Passou a tirar fotos mais nítidas e de melhor resolução.", "O local ficou mais seguro e esteticamente agradável.", "A demanda por moradias e serviços aumentou consideravelmente.", "Ela sentiu uma melhora no seu foco e redução do estresse.", "Teve um melhor desempenho e conquistou o primeiro lugar.", "A conexão ficou mais estável e acessível para os moradores.", "A procura por agasalhos e cobertores aumentou nas lojas.", "Acabou se atrasando para o trabalho na manhã seguinte.", "A satisfação e motivação dos colaboradores aumentaram.", "O local foi evacuado por segurança, e o serviço foi interrompido temporariamente.", "Os alunos começaram a frequentá-la mais e a melhorar seu desempenho acadêmico.", "Ele se sentiu mais produtivo e menos cansado ao final do dia.", "O número de usuários aumentou rapidamente.", "Diversas áreas urbanas e rurais foram afetadas por inundações, forçando muitas famílias a deixarem suas casas temporariamente.", "Em poucos meses, notou uma grande melhora em sua disposição, concentração e níveis de energia, além de perder peso.", "Em menos de um ano, a moral da equipe melhorou, a rotatividade diminuiu, e a produtividade geral aumentou em cerca de 30%.", "Como resultado, houve uma redução significativa na quantidade de resíduos sólidos em aterros e uma maior economia de recursos naturais.", "Em um ano, a produção agrícola voltou a níveis estáveis, e o impacto econômico negativo foi mitigado, beneficiando a população.", "Pela manhã, o dispositivo estava superaquecido e apresentou danos permanentes na bateria, reduzindo sua capacidade de funcionamento.", "Com a revitalização, a praça voltou a ser um local de encontro para moradores, e a segurança da área também melhorou.", "Apesar do atraso, a reforma resultou em um edifício mais seguro, confortável e com um aumento significativo no valor do imóvel.", "A resposta emergencial foi mobilizada rapidamente, com abrigos temporários e ajuda humanitária distribuída para minimizar o impacto nas vítimas.", "A planta murchou completamente e, infelizmente, não conseguiu ser recuperada, tendo que ser substituída.", "Ele ficou impossibilitado de trabalhar por duas semanas.", "A comunidade teve que recorrer a abrigos temporários, e a economia local sofreu uma queda significativa devido à paralisação das atividades comerciais.", "Ficou molhada durante o trajeto para o trabalho.", "A reputação da empresa melhorou, atraindo consumidores conscientes e aumentando as vendas em 20%.", "Sua mãe ficou preocupada com a falta de nutrientes na alimentação dele e decidiu consultar um nutricionista.", "Isso resultou na redução significativa das emissões de carbono e no avanço tecnológico sustentável, beneficiando o meio ambiente globalmente.", "Ela sentiu fome e falta de energia durante a manhã de trabalho.", "As praias ficaram mais limpas e atraentes para turistas, além de promover a conscientização ambiental entre os moradores.", "Finalmente recebeu uma oferta de trabalho em uma empresa renomada, melhorando sua estabilidade financeira.", "Suas peças ganharam destaque em exposições internacionais, ampliando seu reconhecimento e alcance no mercado de arte."]
}

# Crie o DataFrame
df_validation = pd.DataFrame(data)

# Adicionar frases iguais
df_zero = pd.DataFrame({
    'target_transformed': [0.0] * len(data['target_transformed']),
    'content1': df_validation['content1'],
    'content2': df_validation['content1']
})

# Adicionar frases com a ordem contrária
df_reversed = pd.DataFrame({
    'target_transformed': [-1.0] * len(data['target_transformed']),
    'content1': df_validation['content2'],
    'content2': df_validation['content1']
})

# Concatenar os dois DataFrames
df_validation = pd.concat([df_zero, df_validation, df_reversed], ignore_index=True)

df_validation

,target_transformed,content1,content2
0,0.0,A tempestade durou várias horas durante a noite.,A tempestade durou várias horas durante a noite.
1,0.0,Ele estudou intensamente para o exame.,Ele estudou intensamente para o exame.
2,0.0,A fábrica reduziu a emissão de poluentes.,A fábrica reduziu a emissão de poluentes.
3,0.0,Maria começou a praticar exercícios regularmente.,Maria começou a praticar exercícios regularmente.
4,0.0,O motor do carro parou de funcionar.,O motor do carro parou de funcionar.
...,...,...,...
145,-1.0,Isso resultou na redução significativa das emi...,"Após anos de pesquisa e desenvolvimento, a equ..."
146,-1.0,Ela sentiu fome e falta de energia durante a m...,O café da manhã foi esquecido.
147,-1.0,As praias ficaram mais limpas e atraentes para...,A iniciativa comunitária organizou mutirões de...
148,-1.0,Finalmente recebeu uma oferta de trabalho em u...,Ele atualizou seu currículo e participou de vá...


In [17]:
encoding1 = embedding_generator.tokenizer(
			list(df_validation['content1']),
			return_tensors="pt",
			padding=True,  # Padding dinâmico
			truncation=True)

encoding2 = embedding_generator.tokenizer(
			list(df_validation['content2']),
			return_tensors="pt",
			padding=True,  # Padding dinâmico
			truncation=True)

# Converter para o dispositivo
input_ids1 = encoding1.input_ids.to(embedding_generator.device)
attention_mask1 = encoding1.attention_mask.to(embedding_generator.device)
input_ids2 = encoding2.input_ids.to(embedding_generator.device)
attention_mask2 = encoding2.attention_mask.to(embedding_generator.device)

# Obter embeddings
embeddings1 = embedding_generator.weights[input_ids1]
embeddings2 = embedding_generator.weights[input_ids2]

# Concatenar as duas sequências no eixo 1
embeddings = torch.stack([embeddings1, embeddings2], dim=1)  # (batch_size, 2, seq_length, vector_size)

# Calcular comprimentos
lengths1 = attention_mask1.sum(dim=1)
lengths2 = attention_mask2.sum(dim=1)
lengths = torch.stack([lengths1, lengths2], dim=1)  # (batch_size, 2)

# Converter targets
targets = torch.tensor(df_validation['target_transformed'], dtype=torch.float16) # (batch_size,)

print(embeddings.shape,lengths.shape,targets.shape)

torch.Size([150, 2, 61, 2048]) torch.Size([150, 2]) torch.Size([150])


In [ ]:
import torch.nn as nn

def collate_fn(batch):
    embeddings, lengths, targets = zip(*batch)
    return torch.stack(embeddings).to(torch.float32), torch.stack(lengths), torch.stack(targets).to(torch.float32)

In [ ]:
training_dataset = IterableSequenceDataset(db1_path, db2_path)
train_dataloader = DataLoader(
  training_dataset, 
  batch_size=4, 
  num_workers=0,
  collate_fn=collate_fn,
  pin_memory=False,
  shuffle=False,
  #persistent_workers=True
)

validation_dataset = SequenceDataset_val(df_validation)
validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=150,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
    collate_fn=collate_fn
)

In [8]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence

class SequenceToTwoModel(nn.Module):
    def __init__(self, input_size=2048, hidden_size=128, num_layers=3):
        super(SequenceToTwoModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM compartilhado para ambas as sequências
        self.shared_lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.1
        )

        # Camadas totalmente conectadas após concatenar as saídas do LSTM compartilhado para ambas as sequências
        self.fc1 = nn.Linear(hidden_size * 4, 64)  # hidden_size * 2 para cada sequência
        self.leaky_relu = nn.LeakyReLU(inplace=True)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, sequences, lengths):
        """
        Passagem para frente do modelo.

        Args:
            sequences (Tensor): Tensor de embeddings de forma (batch_size, 2, seq_len, input_size)
            lengths (Tensor): Tensor de comprimentos de forma (batch_size, 2)

        Returns:
            Tensor: Previsões do modelo de forma (batch_size,)
        """
        batch_size = sequences.size(0)
        seq_len = sequences.size(2)
        input_size = sequences.size(3)

        # Combinar as duas sequências ao longo da dimensão do batch
        combined_sequences = sequences.view(batch_size * 2, seq_len, input_size)
        combined_lengths = lengths.view(batch_size * 2)

        # Processar as sequências combinadas através do LSTM compartilhado
        lstm_out = self.process_sequence(combined_sequences, combined_lengths)  # (batch_size * 2, hidden_size * 2)

        # Reorganizar a saída para separar as sequências
        lstm_out = lstm_out.view(batch_size, 2, self.hidden_size * 2)

        # Concatenar as saídas das duas sequências
        combined = lstm_out.view(batch_size, -1)  # (batch_size, hidden_size * 4)

        # Passar pelas camadas totalmente conectadas
        out = self.fc1(combined)
        out = self.leaky_relu(out)
        out = self.fc2(out)

        return out.squeeze()
        
    def process_sequence(self, sequence, length):
        """
        Processa uma única sequência através do LSTM compartilhado e retorna o último estado oculto concatenado.

        Args:
            sequence (Tensor): Tensor de forma (batch_size, seq_len, input_size)
            length (Tensor): Tensor de forma (batch_size,)

        Returns:
            Tensor: Saída concatenada do último estado oculto de ambas as direções (batch_size, hidden_size * 2)
        """
        # Ordenar as sequências por comprimento decrescente
        lengths_sorted, sorted_idx = length.sort(0, descending=True)
        sequence_sorted = sequence[sorted_idx]

        # Empacotar as sequências
        packed_seq = pack_padded_sequence(sequence_sorted, lengths_sorted.cpu(), batch_first=True, enforce_sorted=True)

        # Passar pelo LSTM
        packed_output, (h_n, c_n) = self.shared_lstm(packed_seq)

        # h_n: (num_layers * num_directions, batch, hidden_size)
        num_directions = 2  # Bidirecional

        # Reorganizar h_n para (num_layers, num_directions, batch, hidden_size)
        h_n = h_n.view(self.num_layers, num_directions, sequence.size(0), self.hidden_size)

        # Selecionar o último layer
        last_layer_h_n = h_n[-1]  # (num_directions, batch, hidden_size)

        # Concatenar as direções
        gru_out = torch.cat((last_layer_h_n[0], last_layer_h_n[1]), dim=1)  # (batch, hidden_size * 2)

        # Restaurar a ordem original do batch
        _, original_idx = sorted_idx.sort(0, descending=False)
        gru_out = gru_out[original_idx]

        return gru_out

In [9]:
# Inicializa o modelo, critério de perda e otimizador
model = SequenceToTwoModel().to('cuda')
#criterion = nn.MSELoss()
criterion = nn.HuberLoss()

optimizer = torch.optim.Adam(
    model.parameters(), lr=1e-3, weight_decay=1e-6)

# Inicializar o scheduler ReduceLROnPlateau
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=1, verbose=True
)

# Inicializar o contador de passos
step = 0
steps_per_epoch = 10  # Número de batches por época
validation_steps = None  # Se você deseja limitar os passos de validação, defina um número inteiro
num_epochs = 10
# Inicializar o contador de passos
step = 0
from torch.utils.tensorboard import SummaryWriter

/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [10]:
# Inicializar o writer do TensorBoard
writer = SummaryWriter('runs/treinamento_regressao_rnn')

# Envolver o loop de épocas com tqdm para mostrar o progresso das épocas
epoch_progress = tqdm(range(num_epochs), desc='Treinamento', unit='época')

for epoch in epoch_progress:
	epoch_progress.set_description(f'Época {epoch+1}/{num_epochs}')

	print("Treinamento")
	model.train()
	train_loss = 0.0

	# Criar um iterador a partir do dataloader
	train_iterator = iter(train_dataloader)

	# Usar tqdm para o progresso dentro da época
	batch_progress = tqdm(range(steps_per_epoch), desc='Treinando', leave=False, unit='batch')

	for _ in batch_progress:
		try:
			embeddings, lengths, targets = next(train_iterator)
		except StopIteration:
			# Reiniciar o iterador se necessário
			train_iterator = iter(train_dataloader)
			embeddings, lengths, targets = next(train_iterator)

		print(embeddings.shape)
		embeddings = embeddings.to('cuda') # (batch_size, 2, seq_len, vector_size)
		lengths = lengths.to('cuda') # (batch_size, 2)
		targets = targets.to('cuda') # (batch_size,)
	
		optimizer.zero_grad()

		# Forward pass
		outputs = model(embeddings, lengths)
		loss = criterion(outputs, targets)

		# Backward pass e otimização
		loss.backward()
		optimizer.step()

		train_loss += loss.item() * targets.size(0)

		# Registrar a perda de treinamento a cada passo
		writer.add_scalar('Perda_Treinamento_Passo', loss.item(), step)
		step += 1

	avg_train_loss = train_loss / (steps_per_epoch * batch_size)

	# Validação
	model.eval()
	validation_loss = 0.0
	correct_sign = 0
	total = 0

	with torch.no_grad():
		for embeddings, lengths, targets in validation_dataloader:
			print(embeddings.shape)
			embeddings = embeddings.to('cuda') # (batch_size, 2, seq_len, vector_size)
			lengths = lengths.to('cuda') # (batch_size, 2)
			targets = targets.to('cuda') # (batch_size,)

			# Forward pass
			outputs = model(embeddings, lengths)
			loss = criterion(outputs, targets)
			validation_loss += loss.item() * targets.size(0)

			# Considerar apenas os alvos positivos ou negativos
			non_zero_mask = targets != 0
			if non_zero_mask.sum().item() == 0:
				continue  # Pular se não houver alvos diferentes de zero

			predicted_sign = torch.sign(outputs)[non_zero_mask]
			target_sign = torch.sign(targets)[non_zero_mask]

			# Calcular acertos onde os sinais correspondem
			correct_sign += (predicted_sign == target_sign).sum().item()
			total += non_zero_mask.sum().item()

	# Evitar divisão por zero
	if total > 0:
		sign_accuracy = correct_sign / total
	else:
		sign_accuracy = 0.0

	avg_validation_loss = validation_loss / len(validation_dataloader.dataset)

	# Registrar as métricas de validação por época
	writer.add_scalar('Perda_Validação_Epoca', avg_validation_loss, epoch+1)
	writer.add_scalar('Acurácia_Sinal_Validação', sign_accuracy, epoch+1)

	# Passar a perda de validação para o scheduler
	scheduler.step(avg_validation_loss)

	# Registrar a taxa de aprendizado atual (opcional)
	current_lr = optimizer.param_groups[0]['lr']
	writer.add_scalar('Taxa_Aprendizado', current_lr, epoch+1)

	# Atualizar a descrição da barra com as perdas atuais e acurácia do sinal
	epoch_progress.set_postfix({
		'Perda Treinamento': f'{avg_train_loss:.3f}',
		'Perda Validação': f'{avg_validation_loss:.3f}',
		'Acurácia Sinal': f'{sign_accuracy*100:.2f}%',
		'LR': f'{current_lr:.8f}'
	})

	# Liberar cache da GPU após cada época
	torch.cuda.empty_cache()

# Fechar o SummaryWriter ao final do treinamento
writer.close()

Época 1/10:   0%|          | 0/10 [00:00<?, ?época/s] 

Treinamento


_fetch_data
_fetch_data iterrows
collate_fn 1
torch.Size([1, 2, 24, 2048])


/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/torch/nn/modules/loss.py:1100: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta=self.delta)


_fetch_data iterrows
collate_fn 1
torch.Size([1, 2, 12, 2048])
_fetch_data iterrows


collate_fn 1
torch.Size([1, 2, 8, 2048])
_fetch_data iterrows
collate_fn 1
torch.Size([1, 2, 45, 2048])
_fetch_data iterrows
collate_fn 1
torch.Size([1, 2, 12, 2048])
_fetch_data iterrows
collate_fn 1
torch.Size([1, 2, 8, 2048])
_fetch_data iterrows
collate_fn 1
torch.Size([1, 2, 52, 2048])
_fetch_data iterrows
collate_fn 1
torch.Size([1, 2, 50, 2048])
_fetch_data iterrows
collate_fn 1
torch.Size([1, 2, 25, 2048])
_fetch_data iterrows
collate_fn 1
torch.Size([1, 2, 25, 2048])


Época 1/10:   0%|          | 0/10 [00:17<?, ?época/s]


IndexError: index 2 is out of bounds for dimension 0 with size 2

In [9]:
import pytorch_lightning as pl

class SequenceToTwoLightning(pl.LightningModule):
	def __init__(self, input_size=2048, hidden_size=128, num_layers=3, lr=1e-3):
		super(SequenceToTwoLightning, self).__init__()
		self.save_hyperparameters()

		# Instanciar o modelo modificado
		self.model = SequenceToTwoModel(
			input_size=input_size,
			hidden_size=hidden_size,
			num_layers=num_layers
		)

		# Função de perda
		self.criterion = nn.HuberLoss()

	def forward(self, sequences, lengths):
		print("forward",sequences.shape,lengths)
		return self.model(sequences, lengths)

	def training_step(self, batch, batch_idx):
		print("training_step",batch_idx)
		embeddings, lengths, targets = batch  # embeddings: (batch_size, 2, seq_len, vector_size)

		# Mover tensores para o dispositivo correto
		embeddings = embeddings.to(self.device)    # (batch_size, 2, seq_len, vector_size)
		lengths = lengths.to(self.device)    # (batch_size, 2)
		targets = targets.to(self.device)          # (batch_size,)

		# Passagem para frente
		outputs = self(embeddings, lengths)        # (batch_size,)

		# Computar a perda
		loss = self.criterion(outputs, targets)

		# Logar a perda de treinamento
		self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
		return loss

	def validation_step(self, batch, batch_idx):
		print("validation_step",batch_idx)
		embeddings, lengths, targets = batch

		# Mover tensores para o dispositivo correto
		embeddings = embeddings.to(self.device)
		lengths = lengths.to(self.device)
		targets = targets.to(self.device)

		# Passagem para frente
		outputs = self(embeddings, lengths)

		# Computar a perda
		loss = self.criterion(outputs, targets)

		# Logar a perda de validação
		self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

	def configure_optimizers(self):
		optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
		return optimizer

In [10]:
from pytorch_lightning.callbacks import ModelCheckpoint

model = SequenceToTwoLightning()

# Definir callbacks, por exemplo, ModelCheckpoint
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='checkpoints',
    filename='best-checkpoint',
    save_top_k=1,
    mode='min'
)

# Instanciar o Trainer
trainer = pl.Trainer(
    max_epochs=10,
    precision=32,                            # Usa precisão de 16 bits se desejado
    accelerator='gpu',                      
    devices=1 if torch.cuda.is_available() else None,
    callbacks=[checkpoint_callback],
    accumulate_grad_batches=1,               # Ajuste se usar acumulação de gradientes
    log_every_n_steps=50,                    # Ajuste a frequência de log
    deterministic=True,                       # Para reprodutibilidade
    limit_train_batches=10,
    profiler="simple",
    logger=False
)

# Iniciar o treinamento
trainer.fit(
    model,
    train_dataloaders=train_dataloader, #IterableDataset
    val_dataloaders=validation_dataloader #Dataset
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | SequenceToTwoModel | 2.3 M  | train
1 | criterion | HuberLoss          | 0      | train
---------------------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.194     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/pytorch_lightning/core/module.py:516: You called `self.log('val_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


collate_fn 4
validation_step 0
forward torch.Size([4, 2, 61, 2048]) tensor([[15, 15],
        [11, 11],
        [16, 16],
        [13, 13]], device='cuda:0')
collate_fn 4
validation_step 1
forward torch.Size([4, 2, 61, 2048]) tensor([[11, 11],
        [11, 11],
        [10, 10],
        [14, 14]], device='cuda:0')


/home/jadson/anaconda3/envs/pytorch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [11]:
import os
import torch
import torch.nn as nn
from tqdm import tqdm

# Função para Salvar Checkpoints
def save_checkpoint(state, is_best, checkpoint_dir='checkpoints', filename='checkpoint.pth'):
    os.makedirs(checkpoint_dir, exist_ok=True)
    filepath = os.path.join(checkpoint_dir, filename)
    torch.save(state, filepath)
    if is_best:
        best_filepath = os.path.join(checkpoint_dir, 'best_checkpoint.pth')
        torch.save(state, best_filepath)

# Função de Treinamento
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for batch_idx, (embeddings, lengths, targets) in enumerate(tqdm(train_loader, desc="Train")):
        embeddings = embeddings.to(device) # (batch_size, 2, seq_len, vector_size)
        lengths = lengths.to(device) # (batch_size, 2)
        targets = targets.to(device) # (batch_size,)

        optimizer.zero_grad()

        outputs = model(embeddings, lengths)          # (batch_size,)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * embeddings.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

# Função de Validação
def validate(model, device, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for batch_idx, (embeddings, lengths, targets) in enumerate(tqdm(val_loader, desc="Validation")):
            embeddings = embeddings.to(device)
            lengths = lengths.to(device)
            targets = targets.to(device)

            outputs = model(embeddings, lengths)
            loss = criterion(outputs, targets)

            running_loss += loss.item() * embeddings.size(0)

    epoch_loss = running_loss / len(val_loader.dataset)
    return epoch_loss


# Configurações
input_size = 2048
hidden_size = 128
num_layers = 3
lr = 1e-3
batch_size = 32
num_epochs = 10
checkpoint_dir = 'checkpoints'
best_val_loss = float('inf')

# Detectar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

# Instanciar o modelo, mover para o dispositivo
model = SequenceToTwoModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
model.to(device)

# Definir a função de perda e o otimizador
criterion = nn.HuberLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Treinamento
for epoch in range(1, num_epochs + 1):
	print(f"\nEpoch {epoch}/{num_epochs}")

	train_loss = train(model, device, train_dataloader, optimizer, criterion)
	print(f"Perda de Treinamento: {train_loss:.4f}")

	val_loss = validate(model, device, validation_dataloader, criterion)
	print(f"Perda de Validação: {val_loss:.4f}")

	# Checkpointing
	is_best = val_loss < best_val_loss
	if is_best:
		best_val_loss = val_loss
		print(f"Nova melhor perda de validação: {best_val_loss:.4f}. Salvando modelo...")
		save_checkpoint({
			'epoch': epoch,
			'model_state_dict': model.state_dict(),
			'optimizer_state_dict': optimizer.state_dict(),
			'val_loss': val_loss,
		}, is_best, checkpoint_dir=checkpoint_dir)

	# (Opcional) Implementar lógica de early stopping, redução de lr, etc.
	# Exemplo de scheduler para reduzir a taxa de aprendizado se a validação não melhorar
	# scheduler.step(val_loss)

print("Treinamento concluído.")

Usando dispositivo: cuda

Epoch 1/10


Train: 0it [05:23, ?it/s]


KeyboardInterrupt: 

In [ ]:
import os

# Diretório onde os modelos serão salvos
save_dir = './saved_fineweb2'
os.makedirs(save_dir, exist_ok=True)

# Salvando o estado do modelo
model_path = os.path.join(save_dir, 'regression_rnn_model.pth')
torch.save(model_order.state_dict(), model_path)
print(f"Modelo salvo em {model_path}")

# Opcional: Salvando o estado do otimizador
optimizer_path = os.path.join(save_dir, 'optimizer_state.pth')
torch.save(optimizer.state_dict(), optimizer_path)
print(f"Estado do otimizador salvo em {optimizer_path}")

# Opcional: Salvar a época atual para retomar o treinamento
epoch_path = os.path.join(save_dir, 'last_epoch.pth')
torch.save({'epoch': epoch, 'model_state_dict': model_order.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_validation_loss}, epoch_path)
print(f"Checkpoint salvo em {epoch_path}")

In [ ]:
import os
model_path = os.path.join('./saved_fineweb', 'regression_rnn_model.pth')

# Carregar o estado do modelo
model_order.load_state_dict(torch.load(model_path, map_location='cuda'))

# Colocar o modelo em modo de avaliação
#model_order.eval()

In [ ]:
model_order.eval()

sequences1_batch = embedding_generator.text_to_embedding_parallel(df_validation['content1'].tolist(), max_workers=20)
sequences2_batch = embedding_generator.text_to_embedding_parallel(df_validation['content2'].tolist(), max_workers=20)

sequences1_batch = [s.to('cuda') for s in sequences1_batch]
sequences2_batch = [s.to('cuda') for s in sequences2_batch]

targets_batch = targets_batch.to('cuda')

# Realizar a previsão
with torch.no_grad():
	output = model_order(sequences1_batch, sequences2_batch)

	matches = (
		((targets_batch == -1) & (outputs < -1.71)) |  # Regra para -1
		((targets_batch == 0) & ((-1.71 < outputs) & (outputs < 1.71))) |  # Regra para 0
		((targets_batch == 1) & (1.71 < outputs))     # Regra para 1
	)

print(matches.float().mean())
	
# Imprimir os resultados da previsão
for i, pred in enumerate(output):
	print(f"Resultado da previsão para o par {i+1}: {pred.item()}")

In [ ]:
model_order(sequences1_batch[0], sequences2_batch[0])

In [ ]:
sequences1 = embedding_generator.text_to_embedding_parallel(df_validation['content1'].tolist(), max_workers=20)


In [ ]:
from datasets import load_dataset
from scipy.stats import pearsonr, spearmanr

# Carregar o conjunto de dados de avaliação (STS Benchmark)
eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

# Listas para armazenar as pontuações reais e as previsões do modelo
true_scores = []
pred_scores = []

with torch.no_grad():
    for example in eval_dataset:
        sent1 = example['sentence1']
        sent2 = example['sentence2']
        score = example['score']

        # Gerar embeddings para ambas as sentenças
        embedding1 = text_to_embedding(sent1).to('cuda')
        embedding2 = text_to_embedding(sent2).to('cuda')

        # Obter a previsão do modelo
        prediction = model_order([embedding1], [embedding2])
        pred_score = 1 / (1 + abs(prediction.item()))

        # Armazenar as pontuações
        true_scores.append(score)
        pred_scores.append(pred_score)

# Calcular as métricas de correlação
pearson_corr, _ = pearsonr(true_scores, pred_scores)
spearman_corr, _ = spearmanr(true_scores, pred_scores)

print("Similaridade Inversa da Diferença Absoluta")
print(f"Correlação de Pearson: {pearson_corr:.4f}")
print(f"Correlação de Spearman: {spearman_corr:.4f}")

In [ ]:
import torch
from datasets import load_dataset
from scipy.stats import pearsonr, spearmanr
from tqdm import tqdm  # Para acompanhar o progresso

# Carregar o conjunto de dados de avaliação (STS Benchmark)
eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

# Definir o intervalo de valores de k a serem testados
k_values = np.arange(0.001, 100.0, 0.01)
best_k = None
best_pearson = -1  # Inicialização com um valor baixo
best_spearman = -1

# Pré-computar todas as predições para evitar recalcular múltiplas vezes
true_scores = []
predictions = []

with torch.no_grad():
    for example in tqdm(eval_dataset, desc="Processando exemplos"):
        sent1 = example['sentence1']
        sent2 = example['sentence2']
        score = example['score']

        # Gerar embeddings para ambas as sentenças
        embedding1 = text_to_embedding(sent1).to('cuda')
        embedding2 = text_to_embedding(sent2).to('cuda')

        # Obter a previsão do modelo
        prediction = model_order([embedding1], [embedding2])
        predictions.append(prediction.item())

        # Armazenar as pontuações reais
        true_scores.append(score)

true_scores = np.array(true_scores)
predictions = np.array(predictions)

# Iterar sobre os valores de k para encontrar o melhor
for k in tqdm(k_values, desc="Buscando o melhor k"):
    pred_scores = np.exp(-k * np.abs(predictions))
    
    pearson_corr, _ = pearsonr(true_scores, pred_scores)
    spearman_corr, _ = spearmanr(true_scores, pred_scores)
    
    # Verificar se este k é o melhor até agora
    if pearson_corr > best_pearson:
        best_pearson = pearson_corr
        best_spearman = spearman_corr
        best_k = k

print("Similaridade Baseada em Exponencial")
print(f"Melhor valor de k: {best_k}")
print(f"Correlação de Pearson: {best_pearson:.4f}")
print(f"Correlação de Spearman: {best_spearman:.4f}")

In [ ]:
import os
import torch
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
from scipy.stats import pearsonr, spearmanr

# Verificar se CUDA está disponível
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Carregar o modelo SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2").to(device)

# Carregar o conjunto de dados de avaliação (STS Benchmark)
eval_dataset = load_dataset("sentence-transformers/stsb", split="validation")

# Listas para armazenar as pontuações reais e as previsões do modelo
true_scores = []
pred_scores = []

# Obter as sentenças e os escores do conjunto de dados
sentences1 = eval_dataset['sentence1']
sentences2 = eval_dataset['sentence2']
scores = eval_dataset['score']

# Processar uma frase por vez
for i in range(len(eval_dataset)):
    sentence1 = sentences1[i]
    sentence2 = sentences2[i]
    true_score = scores[i]

    # Gerar embeddings para ambas as sentenças individualmente
    embedding1 = model.encode(sentence1, convert_to_tensor=True, device=device, show_progress_bar=False)
    embedding2 = model.encode(sentence2, convert_to_tensor=True, device=device, show_progress_bar=False)

    # Calcular a similaridade cosseno
    cosine_score = util.cos_sim(embedding1, embedding2).item()

    # Escalar a similaridade cosseno de [-1, 1] para [0, 1]
    scaled_score = (cosine_score + 1) / 2

    # Armazenar as pontuações
    true_scores.append(true_score)
    pred_scores.append(scaled_score)

# Calcular as métricas de correlação
pearson_corr, _ = pearsonr(true_scores, pred_scores)
spearman_corr, _ = spearmanr(true_scores, pred_scores)

print(f"Correlação de Pearson: {pearson_corr:.4f}")
print(f"Correlação de Spearman: {spearman_corr:.4f}")

In [ ]:
import os
model_path = os.path.join('./saved_HuberLoss', 'regression_rnn_model.pth')

# Carregar o estado do modelo
#model_order.load_state_dict(torch.load(model_path, map_location='cuda'))

# Colocar o modelo em modo de avaliação
model_order.eval()

# Realizar a previsão
with torch.no_grad():
    output = model_order(
        [text_to_embedding(
            "O aluno estudou para o exame durante a noite.").to('cuda')],
        [text_to_embedding("O resultado foi excelente devido à dedicação.").to('cuda')])

# Imprimir a saída da regressão
print("Resultado da previsão:", output.item())

In [ ]:
del model_order  # Remove a referência ao modelo

# Libera a memória da GPU, se o modelo estiver usando CUDA
torch.cuda.empty_cache()

# Força a coleta de lixo
gc.collect()